In [ ]:
# ! pip install -U langchain-community
# ! pip install -U pypdf
# ! pip install langchain-google-genai
# ! pip install chromadb
!pip install langchain-chroma

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import getpass
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
import google.generativeai as genai

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")

google_api_key = os.environ.get("GOOGLE_API_KEY")

google_embeding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
    )

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, google_embeding)
docs = text_splitter.split_documents(pages)

In [ ]:
docs[-1]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 13, 'page_label': '14'}, page_content='법제처                                                            14                                                       국가법령정보센터\n대한민국헌법\n제5조 이 헌법시행 당시의 법령과 조약은 이 헌법에 위배되지 아니하는 한 그 효력을 지속한다.\n제6조 이 헌법시행 당시에 이 헌법에 의하여 새로 설치될 기관의 권한에 속하는 직무를 행하고 있는 기관은 이 헌법에\n의하여 새로운 기관이 설치될 때까지 존속하며 그 직무를 행한다.')

In [ ]:
query="제70조의 대통령의 임기는?"
docs = db.similarity_search(query)
print(docs[0])
db.similarity_search_with_score(query)

In [ ]:
#save to disk
db2 = Chroma.from_documents(docs, google_embeding, persist_directory="./Chroma_db")

#load from disk
db3=Chroma(persist_directory="./Chroma_db", embedding_function=google_embeding)
result=db3.similarity_search(query)
print(result[0].page_content)

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의


<ipython-input-27-bb29cd640ba7>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db3=Chroma(persist_directory="./Chroma_db", embedding_function=google_embeding)


In [ ]:
import chromadb
#collection을 저장할 경로 지정
client = chromadb.PersistentClient(path="./collection_example")
client.heartbeat()

1737594396011648708

In [ ]:
from chromadb.utils import embedding_functions
import chromadb.utils.embedding_functions as embedding_functions

embedding_function = embedding_functions.GoogleGenerativeAiEmbeddingFunction(google_api_key)

collection=client.get_or_create_collection(name="korean_law", embedding_function=embedding_function)

In [ ]:
collection

Collection(name=korean_law)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages=loader.load_and_split()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)
docs=text_splitter.split_documents(pages)

collection.add(
    ids=[str(i) for i in range(len(docs))],
    documents=[doc.page_content for doc in docs],
    metadatas=[doc.metadata for doc in docs]
)

In [ ]:
Collection=client.get_collection(name="korean_law")

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":1},
)

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":{"$gte":5}},
)

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":1},
    where_document={"$contains":"10"},
)

{'ids': [['4']],
 'embeddings': None,
 'documents': [['법제처                                                            2                                                       국가법령정보센터\n대한민국헌법\n \n       제2장 국민의 권리와 의무\n \n제10조 모든 국민은 인간으로서의 존엄과 가치를 가지며, 행복을 추구할 권리를 가진다. 국가는 개인이 가지는 불가침\n의 기본적 인권을 확인하고 이를 보장할 의무를 진다.\n \n제11조 ①모든 국민은 법 앞에 평등하다. 누구든지 성별ㆍ종교 또는 사회적 신분에 의하여 정치적ㆍ경제적ㆍ사회적ㆍ\n문화적 생활의 모든 영역에 있어서 차별을 받지 아니한다.\n②사회적 특수계급의 제도는 인정되지 아니하며, 어떠한 형태로도 이를 창설할 수 없다.\n③훈장등의 영전은 이를 받은 자에게만 효력이 있고, 어떠한 특권도 이에 따르지 아니한다.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'page': 1,
    'page_label': '2',
    'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf'}]],
 'distances': [[0.27309454930277915]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
import os
import getpass

Chroma().delete_collection()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")
google_api_key = os.environ.get("GOOGLE_API_KEY")

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

google_embeding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
    )
db = Chroma.from_documents(docs, google_embeding)

retreiver = db.as_retriever()
retreiver.invoke("국회의원의 의무")


[Document(id='5a61cf48-6e20-4567-a83c-2d884dde3c7b', metadata={'page': 6, 'page_label': '7', 'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
 Document(id='3d5c9363-ceb5-444d-b22f-4d5cb4dc63b7', metadata={'page': 11, 'page_label': '12', 'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하\n여야 한다.\n②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진

In [ ]:
result_source=db.similarity_search_with_score("국회의원의 의무")
result_r_source=db.similarity_search_with_relevance_scores("국회의원의 의무")
print("[유사청크 1순위]")
print(result_source[0][0].page_content)
print("\n\n[점수]")
print(result_source[0][1])
print(result_r_source[0][1])

[유사청크 1순위]
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의


[점수]
0.5362349152565002
0.6208246551131351


In [ ]:
retreiver=db.as_retriever(search_kwargs={"k":1})
retreiver.get_relevant_documents("국회의원의 의무")

<ipython-input-20-2cd742c3be40>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retreiver.get_relevant_documents("국회의원의 의무")


[Document(id='5a61cf48-6e20-4567-a83c-2d884dde3c7b', metadata={'page': 6, 'page_label': '7', 'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의')]

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade sentence-transformers timm

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
import getpass





Chroma().delete_collection()

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")
# google_api_key = os.environ.get("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API 키를 입력하세요: ")
openai_api_key = os.environ.get("OPENAI_API_KEY")

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

# google_embeding = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=google_api_key
#     )


db = Chroma.from_documents(docs, OpenAIEmbeddings(model='text-embedding-3-small'))

retreiver = db.as_retriever()
retreiver.invoke("국회의원의 의무")

### MMR(Maximal Marginal Relevance) 검색방식

In [11]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages=loader.load_and_split()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)
docs=text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

# retreiver=db.as_retriever(search_type="mmr")
# retreiver.invoke("국회의원의 의무")

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7b7bcbdcb110>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:

GoogleGenerativeAIError: Error embedding content: Timeout of 60.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b7bc82d2110>)